## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 16th Feb. 23rd


In [4]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-21-2025,Condo/Co-op,3140 Holiday Springs Blvd Unit 9-302,Pompano Beach,FL,33063.0,73000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3140-Holiday...,Beaches MLS,F10485343,N,Y,26.265107,-80.232985


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [9]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [10]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [11]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-21    36
2025-02-18    28
2025-02-20    20
2025-02-19    15
2025-02-17     2
Name: count, dtype: int64

In [12]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [13]:
df_top_ten = df2.head(10)

In [14]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
32,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476
46,PAST SALE,2025-02-20,Condo/Co-op,1700 S Ocean Blvd Unit 6D,Lauderdale By The Sea,FL,33062.0,1740000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10474471,N,Y,26.209206,-80.092779
99,PAST SALE,2025-02-18,Condo/Co-op,2200 S Ocean Ln #1102,Fort Lauderdale,FL,33316.0,1520000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,MARMLS,A11563575,N,Y,26.095913,-80.106280
4,PAST SALE,2025-02-18,Condo/Co-op,5100 N Ocean Blvd #1719,Lauderdale By The Sea,FL,33308.0,1200000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11734263,N,Y,26.199954,-80.094697
102,PAST SALE,2025-02-20,Condo/Co-op,333 Las Olas Way #1402,Fort Lauderdale,FL,33301.0,1030000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10425452,N,Y,26.118354,-80.141518
29,PAST SALE,2025-02-18,Condo/Co-op,4040 Galt Ocean Dr #616,Fort Lauderdale,FL,33308.0,915000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Beaches MLS,F10474133,N,Y,26.177340,-80.097077
21,PAST SALE,2025-02-20,Condo/Co-op,3333 NE 32nd Ave #1107,Fort Lauderdale,FL,33308.0,816900.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3333...,Beaches MLS,F10474628,N,Y,26.169556,-80.103127
85,PAST SALE,2025-02-19,Condo/Co-op,1012 N Ocean Blvd #1101,Pompano Beach,FL,33062.0,775000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Beaches MLS,RX-11012408,N,Y,26.247632,-80.086088
35,PAST SALE,2025-02-21,Condo/Co-op,295 N Cortez Dr #295,Pompano Beach,FL,33068.0,620000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/295-N-Cortez...,Beaches MLS,F10476603,N,Y,26.226835,-80.200877
77,PAST SALE,2025-02-21,Condo/Co-op,2030 S Ocean Dr #1606,Hallandale Beach,FL,33009.0,590000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/203...,MARMLS,A11626694,N,Y,25.978359,-80.119546


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

In [20]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,...,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None
1,PAST SALE,2025-02-20,Condo/Co-op,1700 S Ocean Blvd Unit 6D,Lauderdale By The Sea,FL,33062.0,1740000.0,3.0,3.0,...,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Rachel Cruz,Coldwell Banker Realty /Delray Beach,None,None
2,PAST SALE,2025-02-18,Condo/Co-op,2200 S Ocean Ln #1102,Fort Lauderdale,FL,33316.0,1520000.0,2.0,2.0,...,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None
3,PAST SALE,2025-02-18,Condo/Co-op,5100 N Ocean Blvd #1719,Lauderdale By The Sea,FL,33308.0,1200000.0,2.0,2.0,...,-80.094697,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Jeannette Behrens,Coldwell Banker Realty,None,None,None,None,None,None
4,PAST SALE,2025-02-20,Condo/Co-op,333 Las Olas Way #1402,Fort Lauderdale,FL,33301.0,1030000.0,2.0,2.5,...,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,John D'Angelo,D'Angelo Realty Group LLC,Rachel Buemi,D'Angelo Realty Group LLC,Bobbi Ocean,Galleria International Realty,None,None
5,PAST SALE,2025-02-18,Condo/Co-op,4040 Galt Ocean Dr #616,Fort Lauderdale,FL,33308.0,915000.0,2.0,2.0,...,-80.097077,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Susan West,United Realty Group Inc,None,None,Katy Moraskie,Coldwell Banker Realty /Delray Beach,None,None
6,PAST SALE,2025-02-20,Condo/Co-op,3333 NE 32nd Ave #1107,Fort Lauderdale,FL,33308.0,816900.0,2.0,3.0,...,-80.103127,https://www.redfin.com/FL/Fort-Lauderdale/3333...,James Wycoff,Atlantic Properties Int'l Inc,Alexander Nogueira,Atlantic Properties Int'l Inc,Gayle Borden,Coldwell Banker Realty,None,None
7,PAST SALE,2025-02-19,Condo/Co-op,1012 N Ocean Blvd #1101,Pompano Beach,FL,33062.0,775000.0,2.0,2.0,...,-80.086088,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Keriann Worley,"Soaring Real Estate, LLC",Tiffany Goad,Douglas Elliman,Jennifer Vacchetto,Premier Listings,None,None
8,PAST SALE,2025-02-21,Condo/Co-op,295 N Cortez Dr #295,Pompano Beach,FL,33068.0,620000.0,1.0,1.0,...,-80.200877,https://www.redfin.com/FL/Margate/295-N-Cortez...,Sylvie Jolicoeur,Balistreri Real Estate Inc,None,None,Sylvie Jolicoeur,Balistreri Real Estate Inc,None,None
9,PAST SALE,2025-02-21,Condo/Co-op,2030 S Ocean Dr #1606,Hallandale Beach,FL,33009.0,590000.0,2.0,2.0,...,-80.119546,https://www.redfin.com/FL/Hallandale-Beach/203...,Billy Copoloff,Coldwell Banker Realty,None,None,Julia Celie,Coldwell Banker Realty,None,None


## Current Week's Values

In [21]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

101


In [22]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$322,632


In [23]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$269


In [24]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$32,585,800


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
104
Input Previous Week Condo Average Sales Price:
340736
Input Previous Week Condo Average PSF:
270
Input Previous Week Condo Sales total (ex: 198_000_000)
50_100_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [26]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [27]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [28]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [29]:
pd.set_option('display.max_columns',None)

In [30]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,100 Las Olas,2964.0,NaN,2020.0,NaN,742.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None,1,orange


In [31]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [32]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,100 Las Olas,2964.0,NaN,2020.0,NaN,742.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None,1,orange
1,PAST SALE,2025-02-20,Condo/Co-op,1700 S Ocean Blvd Unit 6D,Lauderdale By The Sea,FL,33062.0,1740000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,544.0,2865.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10474471,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Rachel Cruz,Coldwell Banker Realty /Delray Beach,None,None,2,blue
2,PAST SALE,2025-02-18,Condo/Co-op,2200 S Ocean Ln #1102,Fort Lauderdale,FL,33316.0,1520000.0,2.0,2.0,POINT OF AMERICAS,2003.0,NaN,1971.0,NaN,759.0,2170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,MARMLS,A11563575,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue
3,PAST SALE,2025-02-18,Condo/Co-op,5100 N Ocean Blvd #1719,Lauderdale By The Sea,FL,33308.0,1200000.0,2.0,2.0,SEA RANCH CLUB A CONDO,1950.0,NaN,1976.0,NaN,615.0,2048.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11734263,N,Y,26.199954,-80.094697,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Jeannette Behrens,Coldwell Banker Realty,None,None,None,None,None,None,4,blue
4,PAST SALE,2025-02-20,Condo/Co-op,333 Las Olas Way #1402,Fort Lauderdale,FL,33301.0,1030000.0,2.0,2.5,Las Olas River House,1522.0,NaN,2005.0,NaN,677.0,1730.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10425452,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,John D'Angelo,D'Angelo Realty Group LLC,Rachel Buemi,D'Angelo Realty Group LLC,Bobbi Ocean,Galleria International Realty,None,None,5,blue
5,PAST SALE,2025-02-18,Condo/Co-op,4040 Galt Ocean Dr #616,Fort Lauderdale,FL,33308.0,915000.0,2.0,2.0,Ocean Manor Condo,1244.0,NaN,1960.0,NaN,736.0,1511.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Beaches MLS,F10474133,N,Y,26.177340,-80.097077,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Susan West,United Realty Group Inc,None,None,Katy Moraskie,Coldwell Banker Realty /Delray Beach,None,None,6,blue
6,PAST SALE,2025-02-20,Condo/Co-op,3333 NE 32nd Ave #1107,Fort Lauderdale,FL,33308.0,816900.0,2.0,3.0,Il Lugano,1590.0,NaN,2007.0,NaN,514.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3333...,Beaches MLS,F10474628,N,Y,26.169556,-80.103127,https://www.redfin.com/FL/Fort-Lauderdale/3333...,James Wycoff,Atlantic Properties Int'l Inc,Alexander Nogueira,Atlantic Properties Int'l Inc,Gayle Borden,Coldwell Banker Realty,None,None,7,blue
7,PAST SALE,2025-02-19,Condo/Co-op,1012 N Ocean Blvd #1101,Pompano Beach,FL,33062.0,775000.0,2.0,2.0,Century Plaza Condo,1360.0,NaN,1971.0,NaN,570.0,984.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Beaches MLS,RX-11012408,N,Y,26.247632,-80.086088,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Keriann Worley,"Soaring Real Estate, LLC",Tiffany Goad,Douglas Elliman,Jennifer Vacchetto,Premier Listings,None,None,8,blue
8,PAST SALE,2025-02-21,Condo/Co-op,295 N Cortez Dr #295,Pompano Beach,FL,

In [33]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [34]:
m.save('index.html')

## Data snagger

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [36]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,100 Las Olas,2964.0,NaN,2020.0,NaN,742.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [37]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_02242024


# CREATE TEMPLATE 

In [38]:
muni_set = set(df_top_ten['CITY'])

In [39]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [40]:
df_top_ten.reset_index(inplace=True,drop=True)

In [41]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [42]:
top_sale

'100 Las Olas at 100 E Las Olas Blvd #2303 in Fort Lauderdale'

In [43]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [44]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [45]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,100 Las Olas,2964.0,NaN,2020.0,NaN,742.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None,1,orange,100 Las Olas at 100 E Las Olas Blvd #2303 in F...
1,PAST SALE,2025-02-20,Condo/Co-op,1700 S Ocean Blvd Unit 6D,Lauderdale By The Sea,FL,33062.0,1740000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,544.0,2865.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10474471,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Rachel Cruz,Coldwell Banker Realty /Delray Beach,None,None,2,blue,Cristelle at 1700 S Ocean Blvd Unit 6D in Laud...
2,PAST SALE,2025-02-18,Condo/Co-op,2200 S Ocean Ln #1102,Fort Lauderdale,FL,33316.0,1520000.0,2.0,2.0,POINT OF AMERICAS,2003.0,NaN,1971.0,NaN,759.0,2170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,MARMLS,A11563575,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue,POINT OF AMERICAS at 2200 S Ocean Ln #1102 in ...
3,PAST SALE,2025-02-18,Condo/Co-op,5100 N Ocean Blvd #1719,Lauderdale By The Sea,FL,33308.0,1200000.0,2.0,2.0,SEA RANCH CLUB A CONDO,1950.0,NaN,1976.0,NaN,615.0,2048.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11734263,N,Y,26.199954,-80.094697,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Jeannette Behrens,Coldwell Banker Realty,None,None,None,None,None,None,4,blue,SEA RANCH CLUB A CONDO at 5100 N Ocean Blvd #1...
4,PAST SALE,2025-02-20,Condo/Co-op,333 Las Olas Way #1402,Fort Lauderdale,FL,33301.0,1030000.0,2.0,2.5,Las Olas River House,1522.0,NaN,2005.0,NaN,677.0,1730.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10425452,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,John D'Angelo,D'Angelo Realty Group LLC,Rachel Buemi,D'Angelo Realty Group LLC,Bobbi Ocean,Galleria International Realty,None,None,5,blue,Las Olas River House at 333 Las Olas Way #1402...
5,PAST SALE,2025-02-18,Condo/Co-op,4040 Galt Ocean Dr #616,Fort Lauderdale,FL,33308.0,915000.0,2.0,2.0,Ocean Manor Condo,1244.0,NaN,1960.0,NaN,736.0,1511.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Beaches MLS,F10474133,N,Y,26.177340,-80.097077,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Susan West,United Realty Group Inc,None,None,Katy Moraskie,Coldwell Banker Realty /Delray Beach,None,None,6,blue,Ocean Manor Condo at 4040 Galt Ocean Dr #616 i...
6,PAST SALE,2025-02-20,Condo/Co-op,3333 NE 32nd Ave #1107,Fort Lauderdale,FL,33308.0,816900.0,2.0,3.0,Il Lugano,1590.0,NaN,2007.0,NaN,514.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3333...,Beaches MLS,F10474628,N,Y,26.169556,-80.103127,https://www.redfin.com/FL/Fort-Lauderdale/3333...,James Wycoff,Atlantic Properties Int'l Inc,Alexander Nogueira,Atlantic Properties Int'l Inc,Gayle Borden,Coldwell Banker Realty,None,None,7,blue,Il Lugano at 3333 NE 32nd Ave #1107 in Fort La...
7,PAST SALE,2025-02-19,Condo/Co-op,1012 N Ocean Blvd #1101,Pompano Beach,FL,33062.0,775000.0,2.0,2.0,Century Plaza Condo,1360.0,NaN,1971.0,NaN,570.0,984.0

In [46]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 100 Las Olas closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $590,000 to $2,200,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Lauderdale By The Sea, Pompano Beach, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 101 condo sales totaling $32,585,800 million from Feb. 16th to Feb. 23rd. The previous week, brokers closed 104 condo sales totaling $50,100,000.

Last week’s units sold for an average of $322,632, lower than the $340,736 average price from the previous week. The average pr

In [47]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [48]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,100 Las Olas,2964.0,NaN,2020.0,NaN,742.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None,1,orange,100 Las Olas at 100 E Las Olas Blvd #2303 in F...
1,PAST SALE,2025-02-20,Condo/Co-op,1700 S Ocean Blvd Unit 6D,Lauderdale By The Sea,FL,33062.0,1740000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,544.0,2865.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10474471,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Rachel Cruz,Coldwell Banker Realty /Delray Beach,None,None,2,blue,Cristelle at 1700 S Ocean Blvd Unit 6D in Laud...
2,PAST SALE,2025-02-18,Condo/Co-op,2200 S Ocean Ln #1102,Fort Lauderdale,FL,33316.0,1520000.0,2.0,2.0,POINT OF AMERICAS,2003.0,NaN,1971.0,NaN,759.0,2170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,MARMLS,A11563575,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue,POINT OF AMERICAS at 2200 S Ocean Ln #1102 in ...
3,PAST SALE,2025-02-18,Condo/Co-op,5100 N Ocean Blvd #1719,Lauderdale By The Sea,FL,33308.0,1200000.0,2.0,2.0,SEA RANCH CLUB A CONDO,1950.0,NaN,1976.0,NaN,615.0,2048.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11734263,N,Y,26.199954,-80.094697,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Jeannette Behrens,Coldwell Banker Realty,None,None,None,None,None,None,4,blue,SEA RANCH CLUB A CONDO at 5100 N Ocean Blvd #1...
4,PAST SALE,2025-02-20,Condo/Co-op,333 Las Olas Way #1402,Fort Lauderdale,FL,33301.0,1030000.0,2.0,2.5,Las Olas River House,1522.0,NaN,2005.0,NaN,677.0,1730.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10425452,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,John D'Angelo,D'Angelo Realty Group LLC,Rachel Buemi,D'Angelo Realty Group LLC,Bobbi Ocean,Galleria International Realty,None,None,5,blue,Las Olas River House at 333 Las Olas Way #1402...
5,PAST SALE,2025-02-18,Condo/Co-op,4040 Galt Ocean Dr #616,Fort Lauderdale,FL,33308.0,915000.0,2.0,2.0,Ocean Manor Condo,1244.0,NaN,1960.0,NaN,736.0,1511.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Beaches MLS,F10474133,N,Y,26.177340,-80.097077,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Susan West,United Realty Group Inc,None,None,Katy Moraskie,Coldwell Banker Realty /Delray Beach,None,None,6,blue,Ocean Manor Condo at 4040 Galt Ocean Dr #616 i...
6,PAST SALE,2025-02-20,Condo/Co-op,3333 NE 32nd Ave #1107,Fort Lauderdale,FL,33308.0,816900.0,2.0,3.0,Il Lugano,1590.0,NaN,2007.0,NaN,514.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3333...,Beaches MLS,F10474628,N,Y,26.169556,-80.103127,https://www.redfin.com/FL/Fort-Lauderdale/3333...,James Wycoff,Atlantic Properties Int'l Inc,Alexander Nogueira,Atlantic Properties Int'l Inc,Gayle Borden,Coldwell Banker Realty,None,None,7,blue,Il Lugano at 3333 NE 32nd Ave #1107 in Fort La...
7,PAST SALE,2025-02-19,Condo/Co-op,1012 N Ocean Blvd #1101,Pompano Beach,FL,33062.0,775000.0,2.0,2.0,Century Plaza Condo,1360.0,NaN,1971.0,NaN,570.0,984.0

In [51]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 100 Las Olas closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $590,000 to $2,200,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Lauderdale By The Sea, Pompano Beach, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 101 condo sales totaling $32,585,800 million from Feb. 16th to Feb. 23rd. The previous week, brokers closed 104 condo sales totaling $50,100,000.

Last week’s units sold for an average of $322,632, lower than the $340,736 average price from the previous week. The average pr

In [52]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/100-E-Las-Olas-Blvd-33301/unit-2303/home/160370322


In [53]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Pompano-Beach/1700-S-Ocean-Blvd-33062/unit-6D/home/41979337


In [54]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hallandale-Beach/2030-S-Ocean-Dr-33009/unit-1606/home/41964854


In [55]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Margate/295-N-Cortez-Dr-33068/unit-295/home/42073603


In [56]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Margate/295-N-Cortez-Dr-33068/unit-295/home/42073603


In [57]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hallandale-Beach/2030-S-Ocean-Dr-33009/unit-1606/home/41964854


In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-21,Condo/Co-op,100 E Las Olas Blvd #2303,Fort Lauderdale,FL,33301.0,2200000.0,3.0,3.5,100 Las Olas,2964.0,NaN,2020.0,NaN,742.0,3667.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10475768,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Austin Bergman,"Compass Florida, LLC",None,None,Courtney Fuhrmann,Waterfront Properties & Club C,None,None,1,orange,100 Las Olas at 100 E Las Olas Blvd #2303 in F...
1,PAST SALE,2025-02-20,Condo/Co-op,1700 S Ocean Blvd Unit 6D,Lauderdale By The Sea,FL,33062.0,1740000.0,3.0,3.0,Cristelle,3200.0,NaN,1998.0,NaN,544.0,2865.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1700-S...,Beaches MLS,F10474471,N,Y,26.209206,-80.092779,https://www.redfin.com/FL/Pompano-Beach/1700-S...,David Saindon,RE/MAX Experience,Robert Shastany,RE/MAX Experience,Rachel Cruz,Coldwell Banker Realty /Delray Beach,None,None,2,blue,Cristelle at 1700 S Ocean Blvd Unit 6D in Laud...
2,PAST SALE,2025-02-18,Condo/Co-op,2200 S Ocean Ln #1102,Fort Lauderdale,FL,33316.0,1520000.0,2.0,2.0,POINT OF AMERICAS,2003.0,NaN,1971.0,NaN,759.0,2170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,MARMLS,A11563575,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue,POINT OF AMERICAS at 2200 S Ocean Ln #1102 in ...
3,PAST SALE,2025-02-18,Condo/Co-op,5100 N Ocean Blvd #1719,Lauderdale By The Sea,FL,33308.0,1200000.0,2.0,2.0,SEA RANCH CLUB A CONDO,1950.0,NaN,1976.0,NaN,615.0,2048.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11734263,N,Y,26.199954,-80.094697,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Jeannette Behrens,Coldwell Banker Realty,None,None,None,None,None,None,4,blue,SEA RANCH CLUB A CONDO at 5100 N Ocean Blvd #1...
4,PAST SALE,2025-02-20,Condo/Co-op,333 Las Olas Way #1402,Fort Lauderdale,FL,33301.0,1030000.0,2.0,2.5,Las Olas River House,1522.0,NaN,2005.0,NaN,677.0,1730.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Beaches MLS,F10425452,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,John D'Angelo,D'Angelo Realty Group LLC,Rachel Buemi,D'Angelo Realty Group LLC,Bobbi Ocean,Galleria International Realty,None,None,5,blue,Las Olas River House at 333 Las Olas Way #1402...
5,PAST SALE,2025-02-18,Condo/Co-op,4040 Galt Ocean Dr #616,Fort Lauderdale,FL,33308.0,915000.0,2.0,2.0,Ocean Manor Condo,1244.0,NaN,1960.0,NaN,736.0,1511.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Beaches MLS,F10474133,N,Y,26.177340,-80.097077,https://www.redfin.com/FL/Fort-Lauderdale/4040...,Susan West,United Realty Group Inc,None,None,Katy Moraskie,Coldwell Banker Realty /Delray Beach,None,None,6,blue,Ocean Manor Condo at 4040 Galt Ocean Dr #616 i...
6,PAST SALE,2025-02-20,Condo/Co-op,3333 NE 32nd Ave #1107,Fort Lauderdale,FL,33308.0,816900.0,2.0,3.0,Il Lugano,1590.0,NaN,2007.0,NaN,514.0,1757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3333...,Beaches MLS,F10474628,N,Y,26.169556,-80.103127,https://www.redfin.com/FL/Fort-Lauderdale/3333...,James Wycoff,Atlantic Properties Int'l Inc,Alexander Nogueira,Atlantic Properties Int'l Inc,Gayle Borden,Coldwell Banker Realty,None,None,7,blue,Il Lugano at 3333 NE 32nd Ave #1107 in Fort La...
7,PAST SALE,2025-02-19,Condo/Co-op,1012 N Ocean Blvd #1101,Pompano Beach,FL,33062.0,775000.0,2.0,2.0,Century Plaza Condo,1360.0,NaN,1971.0,NaN,570.0,984.0

## Time on Market Calculator

In [60]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 12, 4) ## List (Earlier) date
date2 = datetime(2025, 2, 20) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

78


## Clean CSV for Datawrapper Chart

In [61]:
chart_df = df_top_ten

In [62]:
chart_df = chart_df.fillna(" ")

In [63]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [64]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [65]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [66]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [67]:
chart_df['$/SQUARE FEET'].astype(float)

0    742.0
1    544.0
2    759.0
3    615.0
4    677.0
5    736.0
6    514.0
7    570.0
8    820.0
9    450.0
Name: $/SQUARE FEET, dtype: float64

In [68]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [69]:
csv_date_string = today.strftime("%m_%d_%Y")

In [70]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [71]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-21,100 E Las Olas Blvd #2303 in Fort Lauderdale,"$2,200,000",3,3.5,"2,964",2020,$742,Beaches MLS,F10475768,"Austin Bergman Compass Florida, LLC",Courtney Fuhrmann Waterfront Properties & Club...
1,2025-02-20,1700 S Ocean Blvd Unit 6D in Lauderdale By The...,"$1,740,000",3,3.0,"3,200",1998,$544,Beaches MLS,F10474471,David Saindon RE/MAX Experience Robert Shastan...,Rachel Cruz Coldwell Banker Realty /Delray Bea...
2,2025-02-18,2200 S Ocean Ln #1102 in Fort Lauderdale,"$1,520,000",2,2.0,"2,003",1971,$759,MARMLS,A11563575,Brian Hero One Sotheby's Int'l Realty,Brian Hero One Sotheby's Int'l Realty
3,2025-02-18,5100 N Ocean Blvd #1719 in Lauderdale By The Sea,"$1,200,000",2,2.0,"1,950",1976,$615,MARMLS,A11734263,Jeannette Behrens Coldwell Banker Realty,
4,2025-02-20,333 Las Olas Way #1402 in Fort Lauderdale,"$1,030,000",2,2.5,"1,522",2005,$677,Beaches MLS,F10425452,John D'Angelo D'Angelo Realty Group LLC Rachel...,Bobbi Ocean Galleria International Realty
5,2025-02-18,4040 Galt Ocean Dr #616 in Fort Lauderdale,"$915,000",2,2.0,"1,244",1960,$736,Beaches MLS,F10474133,Susan West United Realty Group Inc,Katy Moraskie Coldwell Banker Realty /Delray B...
6,2025-02-20,3333 NE 32nd Ave #1107 in Fort Lauderdale,"$816,900",2,3.0,"1,590",2007,$514,Beaches MLS,F10474628,James Wycoff Atlantic Properties Int'l Inc Ale...,Gayle Borden Coldwell Banker Realty
7,2025-02-19,1012 N Ocean Blvd #1101 in Pompano Beach,"$775,000",2,2.0,"1,360",1971,$570,Beaches MLS,RX-11012408,"Keriann Worley Soaring Real Estate, LLC Tiffan...",Jennifer Vacchetto Premier Listings
8,2025-02-21,295 N Cortez Dr #295 in Pompano Beach,"$620,000",1,1.0,756,2018,$820,Beaches MLS,F10476603,Sylvie Jolicoeur Balistreri Real Estate Inc,Sylvie Jolicoeur Balistreri Real Estate Inc
9,2025-02-21,2030 S Ocean Dr #1606 in Hallandale Beach,"$590,000",2,2.0,"1,312",1970,$450,MARMLS,A11626694,Billy Copoloff Coldwell Banker Realty,Julia Celie Coldwell Banker Realty
